# SPLADE vs sklearn: Comprehensive Benchmark

This notebook demonstrates the advantages of neural sparse SPLADE classification over traditional sklearn TF-IDF:

1. **Statistical Rigor**: Multi-seed experiments with bootstrap CIs, McNemar's test, effect sizes
2. **GPU Acceleration**: Fused Triton/CUDA kernels with 4-7x speedup
3. **Interpretability**: Semantic explanations via pretrained MLM head
4. **Production Ready**: Save/load, diagnostics, sklearn-compatible API

In [ ]:
import time
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

from src.models import SPLADEClassifier
from src.data import load_classification_data, list_supported_datasets
from src.utils import (
    set_seed,
    bootstrap_ci,
    mcnemar_test,
    paired_t_test,
    effect_size_cohens_d,
    load_stopwords,
)
from src.ops import (
    splade_aggregate,
    flops_reg,
    TRITON_AVAILABLE,
    CUDA_AVAILABLE,
)
from src.ops.splade_kernels import (
    splade_aggregate_pytorch,
    splade_aggregate_triton,
    flops_regularization_pytorch,
    flops_regularization_triton,
)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Triton kernels: {TRITON_AVAILABLE}")
print(f"CUDA C++ kernels: {CUDA_AVAILABLE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"\nSupported datasets: {list_supported_datasets()}")

## 1. GPU Kernel Performance

SPLADE fuses 4 operations (ReLU + log1p + mask + max-pool) into a single kernel, reducing memory traffic by ~4x.

In [ ]:
def benchmark_kernel(fn, *args, warmup=10, iterations=100):
    """Benchmark GPU kernel with proper synchronization."""
    for _ in range(warmup):
        fn(*args)
    torch.cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(iterations):
        fn(*args)
    torch.cuda.synchronize()
    return (time.perf_counter() - start) / iterations * 1000

if torch.cuda.is_available():
    batch_size, seq_len, vocab_size = 32, 128, 30522
    logits = torch.randn(batch_size, seq_len, vocab_size, device='cuda')
    mask = torch.ones(batch_size, seq_len, device='cuda')
    mask[:, -20:] = 0
    
    print(f"SPLADE Aggregation (batch={batch_size}, seq={seq_len}, vocab={vocab_size:,})")
    print("-" * 50)
    
    pytorch_ms = benchmark_kernel(splade_aggregate_pytorch, logits, mask)
    print(f"PyTorch (reference):  {pytorch_ms:.3f} ms")
    
    if TRITON_AVAILABLE:
        triton_ms = benchmark_kernel(splade_aggregate_triton, logits, mask)
        print(f"Triton (fused):       {triton_ms:.3f} ms  ({pytorch_ms/triton_ms:.1f}x faster)")
        
        # Verify correctness
        ref = splade_aggregate_pytorch(logits, mask)
        tri = splade_aggregate_triton(logits, mask)
        print(f"Max numerical diff:   {(ref - tri).abs().max():.2e}")
    
    # FLOPS regularization
    activations = torch.randn(batch_size, vocab_size, device='cuda')
    print(f"\nFLOPS Regularization")
    print("-" * 50)
    
    pytorch_flops_ms = benchmark_kernel(flops_regularization_pytorch, activations)
    print(f"PyTorch (reference):  {pytorch_flops_ms:.3f} ms")
    
    if TRITON_AVAILABLE:
        triton_flops_ms = benchmark_kernel(flops_regularization_triton, activations)
        print(f"Triton (fused):       {triton_flops_ms:.3f} ms  ({pytorch_flops_ms/triton_flops_ms:.1f}x faster)")

## 2. Data Loading

In [ ]:
DATASET = "ag_news"
TRAIN_SIZE = 5000
EPOCHS = 3
SEEDS = [42, 123, 456]

train_texts, train_labels, train_meta = load_classification_data(
    dataset=DATASET, split="train", max_samples=TRAIN_SIZE, seed=42
)
test_texts, test_labels, test_meta = load_classification_data(
    dataset=DATASET, split="test", label_mapping=train_meta['label_mapping']
)

CLASS_NAMES = train_meta['class_names']
NUM_CLASSES = train_meta['num_labels']
NUM_LABELS = 1 if NUM_CLASSES == 2 else NUM_CLASSES

print(f"Dataset: {DATASET}")
print(f"Train: {len(train_texts):,}, Test: {len(test_texts):,}")
print(f"Classes ({NUM_CLASSES}): {CLASS_NAMES}")

## 3. Multi-Seed Experiment with Statistical Analysis

Rigorous evaluation with bootstrap confidence intervals, following NeurIPS best practices.

In [ ]:
sklearn_results = []
splade_results = []
sklearn_all_preds = []
splade_all_preds = []

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Seed {seed}")
    print(f"{'='*60}")
    set_seed(seed)
    
    # sklearn TF-IDF + Logistic Regression
    print("\n[sklearn TF-IDF]")
    t0 = time.time()
    vectorizer = TfidfVectorizer(max_features=30000, ngram_range=(1, 2), sublinear_tf=True)
    X_train = vectorizer.fit_transform(train_texts)
    X_test = vectorizer.transform(test_texts)
    
    lr_clf = LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=seed)
    lr_clf.fit(X_train, train_labels)
    sklearn_time = time.time() - t0
    
    sklearn_preds = lr_clf.predict(X_test)
    sklearn_acc = accuracy_score(test_labels, sklearn_preds)
    sklearn_f1 = f1_score(test_labels, sklearn_preds, average='macro')
    sklearn_sparsity = (1 - X_test.nnz / (X_test.shape[0] * X_test.shape[1])) * 100
    
    sklearn_results.append({'accuracy': sklearn_acc, 'f1': sklearn_f1, 'sparsity': sklearn_sparsity, 'time': sklearn_time})
    sklearn_all_preds.append(sklearn_preds)
    print(f"  Accuracy: {sklearn_acc:.4f}, F1: {sklearn_f1:.4f}, Time: {sklearn_time:.1f}s")
    
    # SPLADE Neural Classifier
    print("\n[SPLADE Neural]")
    splade_clf = SPLADEClassifier(
        num_labels=NUM_LABELS,
        class_names=CLASS_NAMES,
        batch_size=32,
        learning_rate=2e-5,
        flops_lambda=1e-4,
        random_state=seed,
        verbose=True,
    )
    
    t0 = time.time()
    splade_clf.fit(train_texts, train_labels, epochs=EPOCHS)
    splade_time = time.time() - t0
    
    splade_preds = splade_clf.predict(test_texts)
    splade_acc = accuracy_score(test_labels, splade_preds)
    splade_f1 = f1_score(test_labels, splade_preds, average='macro')
    splade_sparsity = splade_clf.get_sparsity(test_texts[:500])
    
    splade_results.append({'accuracy': splade_acc, 'f1': splade_f1, 'sparsity': splade_sparsity, 'time': splade_time})
    splade_all_preds.append(splade_preds)
    print(f"  Accuracy: {splade_acc:.4f}, F1: {splade_f1:.4f}, Sparsity: {splade_sparsity:.1f}%")

## 4. Statistical Comparison

In [ ]:
sklearn_accs = np.array([r['accuracy'] for r in sklearn_results])
splade_accs = np.array([r['accuracy'] for r in splade_results])

print("="*70)
print("STATISTICAL SUMMARY")
print("="*70)

# Bootstrap confidence intervals
sklearn_ci = bootstrap_ci(sklearn_accs, confidence_level=0.95, random_state=42)
splade_ci = bootstrap_ci(splade_accs, confidence_level=0.95, random_state=42)

print(f"\n{'Model':<20} {'Mean Acc':>12} {'Std':>10} {'95% CI':>24}")
print("-" * 70)
print(f"{'sklearn TF-IDF':<20} {sklearn_ci.mean:>12.4f} {sklearn_ci.std:>10.4f} [{sklearn_ci.ci_lower:.4f}, {sklearn_ci.ci_upper:.4f}]")
print(f"{'SPLADE Neural':<20} {splade_ci.mean:>12.4f} {splade_ci.std:>10.4f} [{splade_ci.ci_lower:.4f}, {splade_ci.ci_upper:.4f}]")

# Paired t-test
t_stat, p_value, significant = paired_t_test(splade_accs, sklearn_accs)
print(f"\nPaired t-test: t={t_stat:.3f}, p={p_value:.4f}")
print(f"Significant at α=0.05: {'YES' if significant else 'NO'}")

# Effect size (Cohen's d)
cohens_d = effect_size_cohens_d(splade_accs, sklearn_accs)
effect_interp = "large" if abs(cohens_d) >= 0.8 else "medium" if abs(cohens_d) >= 0.5 else "small" if abs(cohens_d) >= 0.2 else "negligible"
print(f"\nEffect size (Cohen's d): {cohens_d:.3f} ({effect_interp})")
print(f"Mean improvement: {np.mean(splade_accs) - np.mean(sklearn_accs):+.4f}")

In [ ]:
# McNemar's test (using last seed's predictions)
mcnemar_result = mcnemar_test(
    np.array(test_labels),
    sklearn_all_preds[-1],
    np.array(splade_all_preds[-1]),
)

print("McNemar's Test (paired classifier comparison)")
print("-" * 50)
print(f"Chi-squared statistic: {mcnemar_result.statistic:.3f}")
print(f"p-value: {mcnemar_result.p_value:.4f}")
print(f"Significant at α=0.05: {'YES' if mcnemar_result.significant else 'NO'}")
print(f"\nDiscordant pairs:")
print(f"  sklearn correct, SPLADE wrong: {mcnemar_result.model1_better}")
print(f"  SPLADE correct, sklearn wrong: {mcnemar_result.model2_better}")

## 5. Per-Class Performance

In [ ]:
print("sklearn TF-IDF Classification Report:")
print(classification_report(test_labels, sklearn_all_preds[-1], target_names=CLASS_NAMES))

print("\nSPLADE Neural Classification Report:")
print(classification_report(test_labels, splade_all_preds[-1], target_names=CLASS_NAMES))

## 6. Interpretability: Semantic Explanations

SPLADE uses a pretrained MLM head, producing semantically meaningful term weights. TF-IDF produces statistical term frequencies.

In [ ]:
examples = [
    ("Apple stock surged 5% after announcing record iPhone sales and strong quarterly earnings.", "Business"),
    ("The Lakers defeated the Celtics 112-98 in an exciting NBA playoff game last night.", "Sports"),
    ("NASA scientists discovered a new exoplanet that could potentially support liquid water.", "Sci/Tech"),
    ("The United Nations Security Council held an emergency meeting on the refugee crisis.", "World"),
]

feature_names = vectorizer.get_feature_names_out()

for text, expected in examples:
    print(f"\n{'='*70}")
    print(f"Text: \"{text[:65]}...\"")
    print(f"Expected class: {expected}")
    print(f"{'='*70}")
    
    # SPLADE prediction and explanation
    splade_pred_idx = splade_clf.predict([text])[0]
    splade_pred_label = CLASS_NAMES[splade_pred_idx] if CLASS_NAMES else f"Class {splade_pred_idx}"
    splade_probs = splade_clf.predict_proba([text])[0]
    
    print(f"\nSPLADE prediction: {splade_pred_label} (confidence: {max(splade_probs):.1%})")
    print(f"Class probabilities: {[f'{p:.1%}' for p in splade_probs]}")
    
    # Top terms comparison
    splade_terms = splade_clf.explain(text, top_k=8, filter_stopwords=True, filter_subwords=True)
    
    tfidf_vec = vectorizer.transform([text])
    tfidf_weights = tfidf_vec.toarray()[0]
    tfidf_top_idx = tfidf_weights.argsort()[-8:][::-1]
    tfidf_terms = [(feature_names[i], tfidf_weights[i]) for i in tfidf_top_idx if tfidf_weights[i] > 0]
    
    print(f"\n{'TF-IDF terms':<35} {'SPLADE terms':<35}")
    print("-" * 70)
    for i in range(max(len(tfidf_terms), len(splade_terms))):
        tfidf_str = f"{tfidf_terms[i][0]} ({tfidf_terms[i][1]:.3f})" if i < len(tfidf_terms) else ""
        splade_str = f"{splade_terms[i][0]} ({splade_terms[i][1]:.3f})" if i < len(splade_terms) else ""
        print(f"{tfidf_str:<35} {splade_str:<35}")

In [ ]:
# Full explanation with visual bars
print("\nDetailed SPLADE Explanation:")
splade_clf.print_explanation(
    "Breaking: Tech giant Microsoft announces $10 billion investment in artificial intelligence research.",
    top_k=15,
    filter_stopwords=True,
    filter_subwords=True,
)

## 7. MLM Head Diagnostics

Verify that the pretrained MLM head weights are loaded correctly for semantic interpretability.

In [ ]:
stats = splade_clf.diagnose_mlm_head()

print("MLM Head Diagnostic")
print("-" * 40)
print(f"Pretrained flag: {stats['mlm_pretrained_flag']}")
print(f"Weight mean: {stats['mean']:.4f}")
print(f"Weight std: {stats['std']:.4f}")
print(f"Likely pretrained: {stats['likely_pretrained']}")
print(f"\nNote: Pretrained MLM has std ~0.047, random init has std ~0.021")

## 8. Model Persistence

In [ ]:
import os
import tempfile

# Save model
with tempfile.TemporaryDirectory() as tmpdir:
    model_path = os.path.join(tmpdir, "splade_model.pth")
    splade_clf.save(model_path)
    file_size = os.path.getsize(model_path) / (1024 * 1024)
    print(f"Model saved: {model_path}")
    print(f"File size: {file_size:.1f} MB")
    
    # Load model
    loaded_clf = SPLADEClassifier.load(model_path)
    print(f"Model loaded successfully")
    
    # Verify predictions match
    sample_texts = test_texts[:100]
    original_preds = splade_clf.predict(sample_texts)
    loaded_preds = loaded_clf.predict(sample_texts)
    match = sum(o == l for o, l in zip(original_preds, loaded_preds)) / len(original_preds)
    print(f"Prediction match: {match:.1%}")

## 9. Sparsity Analysis

In [ ]:
# Get sparse vectors
sample_size = 500
splade_vectors = splade_clf.transform(test_texts[:sample_size])

print("Sparsity Analysis")
print("-" * 50)
print(f"Vector shape: {splade_vectors.shape}")
print(f"Vocabulary size: {splade_vectors.shape[1]:,}")

# Per-document statistics
nonzero_per_doc = (splade_vectors.abs() > 1e-6).sum(dim=1).float()
sparsity_per_doc = (1 - nonzero_per_doc / splade_vectors.shape[1]) * 100

print(f"\nPer-document non-zero terms:")
print(f"  Mean: {nonzero_per_doc.mean():.0f}")
print(f"  Std: {nonzero_per_doc.std():.0f}")
print(f"  Min: {nonzero_per_doc.min():.0f}")
print(f"  Max: {nonzero_per_doc.max():.0f}")

print(f"\nSparsity: {sparsity_per_doc.mean():.1f}% ± {sparsity_per_doc.std():.1f}%")

# Compare with TF-IDF
tfidf_sample = vectorizer.transform(test_texts[:sample_size])
tfidf_nonzero = tfidf_sample.nnz / sample_size
tfidf_sparsity = (1 - tfidf_sample.nnz / (sample_size * tfidf_sample.shape[1])) * 100

print(f"\nComparison:")
print(f"  SPLADE avg non-zero: {nonzero_per_doc.mean():.0f} / {splade_vectors.shape[1]:,}")
print(f"  TF-IDF avg non-zero: {tfidf_nonzero:.0f} / {tfidf_sample.shape[1]:,}")

## 10. Final Summary

In [ ]:
print("="*70)
print("BENCHMARK SUMMARY")
print("="*70)
print(f"\nDataset: {DATASET}")
print(f"Train size: {len(train_texts):,}")
print(f"Test size: {len(test_texts):,}")
print(f"Seeds: {SEEDS}")
print(f"SPLADE epochs: {EPOCHS}")

print(f"\n{'Metric':<25} {'sklearn TF-IDF':>18} {'SPLADE Neural':>18}")
print("-" * 65)
print(f"{'Accuracy (mean ± std)':<25} {sklearn_ci.mean:>10.4f} ± {sklearn_ci.std:.4f} {splade_ci.mean:>10.4f} ± {splade_ci.std:.4f}")
print(f"{'95% CI':<25} [{sklearn_ci.ci_lower:.4f}, {sklearn_ci.ci_upper:.4f}] [{splade_ci.ci_lower:.4f}, {splade_ci.ci_upper:.4f}]")

sklearn_f1_mean = np.mean([r['f1'] for r in sklearn_results])
splade_f1_mean = np.mean([r['f1'] for r in splade_results])
print(f"{'F1 (macro)':<25} {sklearn_f1_mean:>18.4f} {splade_f1_mean:>18.4f}")

sklearn_sparsity_mean = np.mean([r['sparsity'] for r in sklearn_results])
splade_sparsity_mean = np.mean([r['sparsity'] for r in splade_results])
print(f"{'Sparsity':<25} {sklearn_sparsity_mean:>17.1f}% {splade_sparsity_mean:>17.1f}%")

print(f"\nStatistical significance:")
print(f"  Paired t-test p-value: {p_value:.4f} ({'significant' if significant else 'not significant'})")
print(f"  Cohen's d effect size: {cohens_d:.3f} ({effect_interp})")
print(f"  McNemar's test p-value: {mcnemar_result.p_value:.4f}")

print(f"\nKey advantages of SPLADE:")
print(f"  ✓ Accuracy improvement: {np.mean(splade_accs) - np.mean(sklearn_accs):+.4f}")
print(f"  ✓ Semantic interpretability via pretrained MLM head")
print(f"  ✓ GPU-accelerated with fused Triton/CUDA kernels")
print(f"  ✓ sklearn-compatible API (fit/predict/transform)")